In [1]:
import json
import time
import numpy
import pandas
import sqlalchemy
from data.flat import load

import datetime
import itertools
from data.flat import add_lag, call_them_all, consequentive_lagger, consequentive_pcter, to_date, fill_all, minute_offset

KeyboardInterrupt: 

In [ ]:
# k = 'C:/Users/MainUser/Desktop/AlphaVantage.txt'
k = 'C:/Users/MainUser/Desktop/OpenAPI_sandbox.txt'
crs = open(k, "r")
for columns in ( raw.strip().split() for raw in crs ):
    api_key = columns[0]

In [ ]:
target_quotes = ['MSFT']
news_horizon = 100
effect_horizon = 1
max_quotes_lag = 10

In [ ]:
d = './data/data/rex.xlsx'
data = pandas.read_excel(d)

In [ ]:
# data = data[data['source'] != 'HXK']

In [ ]:
data

In [ ]:
newstitle_frame = data[['id', 'time', 'title']]
lag_markers = list(itertools.product(newstitle_frame['id'].values, numpy.array(numpy.arange(news_horizon - 1)) + 1))
lag_markers = pandas.DataFrame(data=lag_markers, columns=['id', 'lag'])
newstitle_frame = newstitle_frame.merge(right=lag_markers, left_on=['id'], right_on=['id'])

In [ ]:
newstitle_frame

In [ ]:
newstitle_frame['time'] = pandas.to_datetime(newstitle_frame['time'])
newstitle_frame['news_time'] = newstitle_frame['time'].copy()

In [ ]:
# newstitle_frame['time'] = newstitle_frame.apply(func=add_lag, axis=1)
newstitle_frame['time'] = newstitle_frame['lag'].apply(func=minute_offset)
newstitle_frame['time'] = newstitle_frame['news_time'] + newstitle_frame['time']

In [ ]:
beginning_date, ending_date = newstitle_frame['time'].min() - pandas.DateOffset(
    minutes=(max_quotes_lag + effect_horizon)), newstitle_frame['time'].max()
# beginning_date = datetime.datetime.combine(beginning_date, datetime.datetime.min.time())
# ending_date = datetime.datetime.combine(ending_date, datetime.datetime.min.time())

In [ ]:
quotes_data = await call_them_all(tickers=target_quotes,
                                  start_date=beginning_date, end_date=ending_date,
                                  token=api_key)

In [ ]:
quotes_data

In [ ]:
quotes_data = quotes_data.set_index(keys=['ticker', 'time'])
quotes_data = quotes_data.sort_index(ascending=True)

quotes_data = fill_all(frame=quotes_data, freq='T', zero_index_name='ticker', first_index_name='time')

In [ ]:
quotes_data

In [ ]:
quotes_data = consequentive_lagger(frame=quotes_data, n_lags=effect_horizon, suffix='_HOZ')

In [20]:
quotes_data

open_HOZ0  close_HOZ0  high_HOZ0  low_HOZ0  \
ticker time                                                                    
MSFT   2019-08-29 22:50:00+00:00     138.02      138.02     138.02    138.02   
       2019-08-29 22:51:00+00:00        NaN         NaN        NaN       NaN   
       2019-08-29 22:52:00+00:00        NaN         NaN        NaN       NaN   
       2019-08-29 22:53:00+00:00        NaN         NaN        NaN       NaN   
       2019-08-29 22:54:00+00:00        NaN         NaN        NaN       NaN   
...                                     ...         ...        ...       ...   
       2019-10-04 21:53:00+00:00        NaN         NaN        NaN       NaN   
       2019-10-04 21:54:00+00:00        NaN         NaN        NaN       NaN   
       2019-10-04 21:55:00+00:00     138.15      138.15     138.15    138.15   
       2019-10-04 21:56:00+00:00     138.15      138.15     138.15    138.15   
       2019-10-04 21:57:00+00:00     138.15      138.15     138.15    138.15   

                                 volume_HOZ0  open_HOZ1  close_HOZ1  \
ticker time                                                           
MSFT   2019-08-29 22:50:00+00:00          12        NaN         NaN   
       2019-08-29 22:51:00+00:00         NaN     138.02      138.02   
       2019-08-29 22:52:00+00:00         NaN        NaN         NaN   
       2019-08-29 22:53:00+00:00         NaN        NaN         NaN   
       2019-08-29 22:54:00+00:00         NaN        NaN         NaN   
...                                      ...        ...         ...   
       2019-10-04 21:53:00+00:00         NaN        NaN         NaN   
       2019-10-04 21:54:00+00:00         NaN        NaN         NaN   
       2019-10-04 21:55:00+00:00           6        NaN         NaN   
       2019-10-04 21:56:00+00:00           1     138.15      138.15   
       2019-10-04 21:57:00+00:00          14     138.15      138.15   

                                  high_HOZ1  low_HOZ1 volume_HOZ1  
ticker time                                                        
MSFT   2019-08-29 22:50:00+00:00        NaN       NaN         NaN  
       2019-08-29 22:51:00+00:00     138.02    138.02          12  
       2019-08-29 22:52:00+00:00        NaN       NaN         NaN  
       2019-08-29 22:53:00+00:00        NaN       NaN         NaN  
       2019-08-29 22:54:00+00:00        NaN       NaN         NaN  
...                                     ...       ...         ...  
       2019-10-04 21:53:00+00:00        NaN       NaN         NaN  
       2019-10-04 21:54:00+00:00        NaN       NaN         NaN  
       2019-10-04 21:55:00+00:00        NaN       NaN         NaN  
       2019-10-04 21:56:00+00:00     138.15    138.15           6  
       2019-10-04 21:57:00+00:00     138.15    138.15           1  

[51788 rows x 10 columns]

In [21]:
quotes_data = consequentive_pcter(frame=quotes_data, horizon=1)

In [22]:
quotes_data

open_HOZ0_PCT1  close_HOZ0_PCT1  \
ticker time                                                         
MSFT   2019-08-29 22:50:00+00:00             NaN              NaN   
       2019-08-29 22:51:00+00:00             NaN              NaN   
       2019-08-29 22:52:00+00:00             NaN              NaN   
       2019-08-29 22:53:00+00:00             NaN              NaN   
       2019-08-29 22:54:00+00:00             NaN              NaN   
...                                          ...              ...   
       2019-10-04 21:53:00+00:00             NaN              NaN   
       2019-10-04 21:54:00+00:00             NaN              NaN   
       2019-10-04 21:55:00+00:00             NaN              NaN   
       2019-10-04 21:56:00+00:00             0.0              0.0   
       2019-10-04 21:57:00+00:00             0.0              0.0   

                                  high_HOZ0_PCT1  low_HOZ0_PCT1  \
ticker time                                                       
MSFT   2019-08-29 22:50:00+00:00             NaN            NaN   
       2019-08-29 22:51:00+00:00             NaN            NaN   
       2019-08-29 22:52:00+00:00             NaN            NaN   
       2019-08-29 22:53:00+00:00             NaN            NaN   
       2019-08-29 22:54:00+00:00             NaN            NaN   
...                                          ...            ...   
       2019-10-04 21:53:00+00:00             NaN            NaN   
       2019-10-04 21:54:00+00:00             NaN            NaN   
       2019-10-04 21:55:00+00:00             NaN            NaN   
       2019-10-04 21:56:00+00:00             0.0            0.0   
       2019-10-04 21:57:00+00:00             0.0            0.0   

                                 volume_HOZ0_PCT1  open_HOZ1_PCT1  \
ticker time                                                         
MSFT   2019-08-29 22:50:00+00:00              NaN             NaN   
       2019-08-29 22:51:00+00:00              NaN             NaN   
       2019-08-29 22:52:00+00:00              NaN             NaN   
       2019-08-29 22:53:00+00:00              NaN             NaN   
       2019-08-29 22:54:00+00:00              NaN             NaN   
...                                           ...             ...   
       2019-10-04 21:53:00+00:00              NaN             NaN   
       2019-10-04 21:54:00+00:00              NaN             NaN   
       2019-10-04 21:55:00+00:00              NaN             NaN   
       2019-10-04 21:56:00+00:00        -0.833333             NaN   
       2019-10-04 21:57:00+00:00               13             0.0   

                                  close_HOZ1_PCT1  high_HOZ1_PCT1  \
ticker time                                                         
MSFT   2019-08-29 22:50:00+00:00              NaN             NaN   
       2019-08-29 22:51:00+00:00              NaN             NaN   
       2019-08-29 22:52:00+00:00              NaN             NaN   
       2019-08-29 22:53:00+00:00              NaN             NaN   
       2019-08-29 22:54:00+00:00              NaN             NaN   
...                                           ...             ...   
       2019-10-04 21:53:00+00:00              NaN             NaN   
       2019-10-04 21:54:00+00:00              NaN             NaN   
       2019-10-04 21:55:00+00:00              NaN             NaN   
       2019-10-04 21:56:00+00:00              NaN             NaN   
       2019-10-04 21:57:00+00:00              0.0             0.0   

                                  low_HOZ1_PCT1 volume_HOZ1_PCT1  
ticker time                                                       
MSFT   2019-08-29 22:50:00+00:00            NaN              NaN  
       2019-08-29 22:51:00+00:00            NaN              NaN  
       2019-08-29 22:52:00+00:00            NaN              NaN  
       2019-08-29 22:53:00+00:00            NaN              NaN  
       2019-08-29 22:54:00+00:00            NaN              NaN  

In [23]:
quotes_data = quotes_data.reset_index()
# quotes_data['time'] = quotes_data['time'].apply(func=to_date)

In [24]:
quotes_data

,ticker,time,open_HOZ0_PCT1,close_HOZ0_PCT1,high_HOZ0_PCT1,low_HOZ0_PCT1,volume_HOZ0_PCT1,open_HOZ1_PCT1,close_HOZ1_PCT1,high_HOZ1_PCT1,low_HOZ1_PCT1,volume_HOZ1_PCT1
0,MSFT,2019-08-29 22:50:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSFT,2019-08-29 22:51:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MSFT,2019-08-29 22:52:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MSFT,2019-08-29 22:53:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MSFT,2019-08-29 22:54:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
51783,MSFT,2019-10-04 21:53:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51784,MSFT,2019-10-04 21:54:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51785,MSFT,2019-10-04 21:55:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51786,MSFT,2019-10-04 21:56:00+00:00,0.0,0.0,0.0,0.0,-0.833333,NaN,NaN,NaN,NaN,NaN


In [25]:
newstitle_frame['time'] = pandas.to_datetime(newstitle_frame['time'])
quotes_data['time'] = pandas.to_datetime(quotes_data['time'])

In [26]:
qd_tz = quotes_data.loc[0, 'time'].tz

def fix_tz(x):
    return x.tz_localize(tz=qd_tz)

newstitle_frame['time'] = newstitle_frame['time'].apply(func=fix_tz)

In [27]:
def fixit(x):
    return x.ceil(freq='T')

In [28]:
quotes_data['time'] = quotes_data['time'].apply(func=fixit)
newstitle_frame['time'] = newstitle_frame['time'].apply(func=fixit)

In [29]:
with open('E:/InverseStation/terminator_panel/users.json') as f:
    users = json.load(f)

user, password = users['justiciar']['user'], users['justiciar']['password']

with open('E:/InverseStation/terminator_panel/servers.json') as f:
    users = json.load(f)

host, port = users['GOLA']['host'], users['GOLA']['port']

dbname = 'tempbox'

In [30]:
connection_string = "postgresql+psycopg2://{}:{}@{}:{}/{}".format(user, password, host, port, dbname)
engine = sqlalchemy.create_engine(connection_string)
connection = engine.connect()

In [31]:
quotes_data.to_sql(name='quotes_data', con=connection, if_exists='replace', index=False)
newstitle_frame.to_sql(name='newstitle_frame', con=connection, if_exists='replace', index=False)

In [32]:
query = """
SELECT RS.*
FROM
    (SELECT NF."id"
         , NF.title
         , NF."lag"
         , NF.news_time
         , QD.*
    FROM
    public.newstitle_frame AS NF
    FULL OUTER JOIN
    public.quotes_data AS QD
    ON NF."time" = QD."time") AS RS
WHERE 37 = 37
;
"""

result = pandas.read_sql(sql=query, con=connection)

In [35]:
result[~result['title'].isna() & ~result['open_HOZ0_PCT1'].isna()]

,id,title,lag,news_time,ticker,time,open_HOZ0_PCT1,close_HOZ0_PCT1,high_HOZ0_PCT1,low_HOZ0_PCT1,volume_HOZ0_PCT1,open_HOZ1_PCT1,close_HOZ1_PCT1,high_HOZ1_PCT1,low_HOZ1_PCT1,volume_HOZ1_PCT1
2294,161585.0,2019,87.0,2019-08-30 12:58:34.679,MSFT,2019-08-30 14:26:00+00:00,-0.000145,-0.000073,-0.000580,-0.000290,0.186975,0.000218,-0.000218,0.000218,0.000145,0.888550
3571,161585.0,2019,88.0,2019-08-30 12:58:34.679,MSFT,2019-08-30 14:27:00+00:00,0.000000,-0.001016,-0.000073,-0.001234,0.484012,-0.000145,-0.000073,-0.000580,-0.000290,0.186975
4233,161585.0,2019,90.0,2019-08-30 12:58:34.679,MSFT,2019-08-30 14:29:00+00:00,-0.000073,0.000582,0.000436,0.000145,-0.067559,-0.001162,-0.000218,-0.001016,0.000073,-0.442034
5093,161585.0,2019,91.0,2019-08-30 12:58:34.679,MSFT,2019-08-30 14:30:00+00:00,0.000436,0.000073,0.000726,0.000509,0.030205,-0.000073,0.000582,0.000436,0.000145,-0.067559
5958,161585.0,2019,93.0,2019-08-30 12:58:34.679,MSFT,2019-08-30 14:32:00+00:00,0.001017,0.000653,0.000871,0.001090,0.297876,0.000145,0.001017,0.000145,0.000000,-0.411548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930326,390447.0,Trump claims whistleblower source committed 't...,9.0,2019-09-26 21:43:46.795,MSFT,2019-09-26 21:53:00+00:00,-0.000143,0.000000,0.000000,-0.000143,6.250000,0.000143,0.000143,0.000143,0.000143,-0.913043
930373,390564.0,"From earbuds to eyeglasses, Amazon wants Alexa...",6.0,2019-09-26 21:46:41.224,MSFT,2019-09-26 21:53:00+00:00,-0.000143,0.000000,0.000000,-0.000143,6.250000,0.000143,0.000143,0.000143,0.000143,-0.913043
930423,390584.0,Saudi crown prince on Khashoggi killing: ‘I ge...,6.0,2019-09-26 21:46:49.527,MSFT,2019-09-26 21:53:00+00:00,-0.000143,0.000000,0.000000,-0.000143,6.250000,0.000143,0.000143,0.000143,0.000143,-0.913043
930472,390604.0,Naval base in Syria cements Russia’s Mediterra...,6.0,2019-09-26 21:46:49.527,MSFT,2019-09-26 21:53:00+00:00,-0.000143,0.000000,0.000000,-0.000143,6.250000,0.000143,0.000143,0.000143,0.000143,-0.913043


In [ ]:
# https://stackoverflow.com/questions/47386405/memoryerror-when-i-merge-two-pandas-data-frames

In [36]:
the_data = quotes_data.merge(right=newstitle_frame, left_on='time', right_on='target_date', how='outer')

MemoryError: Unable to allocate 931. MiB for an array with shape (122038245,) and data type int64

In [23]:
newstitle_frame['time']

0       2019-08-29 22:36:30.057
1       2019-08-29 22:36:30.057
2       2019-08-29 22:36:30.057
3       2019-08-29 22:36:30.057
4       2019-08-29 22:36:30.057
                  ...          
86620   2019-10-05 06:28:58.138
86621   2019-10-05 06:28:58.138
86622   2019-10-05 06:28:58.138
86623   2019-10-05 06:28:58.138
86624   2019-10-05 06:28:58.138
Name: time, Length: 86625, dtype: datetime64[ns]

In [41]:
the_data

,index,ticker,time_x,open_HOZ0_PCT1,close_HOZ0_PCT1,high_HOZ0_PCT1,low_HOZ0_PCT1,volume_HOZ0_PCT1,open_HOZ1_PCT1,close_HOZ1_PCT1,high_HOZ1_PCT1,low_HOZ1_PCT1,volume_HOZ1_PCT1,id,time_y,title,lag,target_date
0,4859,MSFT,2019-08-30,-0.001811,0.000362,0.000362,-0.001811,-0.75,0.000798,0.000798,0.000798,0.000798,11.0,141465,2019-08-29 22:36:30.057,"Shares rally, bond yields rise as China fuels ...",1,2019-08-30
1,4859,MSFT,2019-08-30,-0.001811,0.000362,0.000362,-0.001811,-0.75,0.000798,0.000798,0.000798,0.000798,11.0,141466,2019-08-29 22:36:30.057,"Argentine bond prices fall, country risk soars...",1,2019-08-30
2,4859,MSFT,2019-08-30,-0.001811,0.000362,0.000362,-0.001811,-0.75,0.000798,0.000798,0.000798,0.000798,11.0,141467,2019-08-29 22:36:30.057,Huawei under probe by U.S. prosecutors over ne...,1,2019-08-30
3,4859,MSFT,2019-08-30,-0.001811,0.000362,0.000362,-0.001811,-0.75,0.000798,0.000798,0.000798,0.000798,11.0,141468,2019-08-29 22:36:30.057,"As U.S. Treasuries sizzle, some bond investors...",1,2019-08-30
4,4859,MSFT,2019-08-30,-0.001811,0.000362,0.000362,-0.001811,-0.75,0.000798,0.000798,0.000798,0.000798,11.0,141469,2019-08-29 22:36:30.057,"China trade comments lift Wall Street as tech,...",1,2019-08-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30837427,21549,MSFT,2019-10-11,0.000357,0.000357,0.000357,0.000357,11.00,-0.000714,-0.000714,-0.000714,-0.000714,0.0,406845,2019-10-05 06:04:32.200,U.S. unemployment rate hits 3.5%; job growth m...,6,2019-10-11
30837428,21549,MSFT,2019-10-11,0.000357,0.000357,0.000357,0.000357,11.00,-0.000714,-0.000714,-0.000714,-0.000714,0.0,406846,2019-10-05 06:04:32.200,Instant View: U.S. September payrolls lower th...,6,2019-10-11
30837429,21549,MSFT,2019-10-11,0.000357,0.000357,0.000357,0.000357,11.00,-0.000714,-0.000714,-0.000714,-0.000714,0.0,406895,2019-10-05 06:12:19.239,Hong Kong metro stays shut after night of viol...,6,2019-10-11
30837430,21549,MSFT,2019-10-11,0.000357,0.000357,0.000357,0.000357,11.00,-0.000714,-0.000714,-0.000714,-0.000714,0.0,406995,2019-10-05 06:28:54.713,UPDATE 3-PayPal becomes first member to exit F...,6,2019-10-11
